In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

from f1tenth_gym_jax import make
import jax
import jax.numpy as jnp

# jax.config.update("jax_debug_nans", True)

from f1tenth_gym_jax.envs.utils import Param

In [2]:
num_agents = 10
num_envs = 100

In [3]:
env = make("default_v0", num_agents=num_agents, params=Param(produce_scans=False))

2024-10-31 11:23:24.673315: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.6.77. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [4]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)

In [5]:
obsv, env_state = jax.vmap(env.reset)(reset_rng)

/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:6217: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  start = asarray(start, dtype=computation_dtype)
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:6218: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  stop = asarray(stop, dtype=computation_dtype)
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.

In [6]:
rng, _rng = jax.random.split(rng)
action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

In [7]:
rng, _rng = jax.random.split(rng)
step_rng = jax.random.split(_rng, num_envs)
obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

In [8]:
def env_init(rng):
    rng, _rng = jax.random.split(rng)
    reset_rng = jax.random.split(_rng, num_envs)
    obsv, env_state = jax.vmap(env.reset)(reset_rng)
    return (env_state, obsv, rng)

In [9]:
def env_step(runner_state, unused):
    env_state, last_obs, rng = runner_state

    rng, _rng = jax.random.split(rng)
    action_rng = jax.random.split(_rng, num_envs*num_agents).reshape((num_agents, num_envs, -1))
    actions = {k: jax.vmap(env.action_space(k).sample)(action_rng[i]) for i, k in enumerate(env.agents)}

    rng, _rng = jax.random.split(rng)
    step_rng = jax.random.split(_rng, num_envs)
    obsv, env_state, _, _, info = jax.vmap(env.step)(step_rng, env_state, actions)

    runner_state = (env_state, obsv, rng)
    return runner_state, None

In [10]:
import time

In [14]:
tick = time.time()
for _ in range(100):
    runner_state = env_init(rng)
    runner_state = jax.lax.scan(env_step, runner_state, None, length=100)
tock = time.time()

In [15]:
sps = 100 * 100 * num_envs / (tock - tick)

In [16]:
sps

272667.3581020087